# Automação Web e Busca de Informações com Python

Desenvolvido pelos alunos: Rafael Kobayashi Santos, Victor Calixto, Gabriel Borges, Gustava Marques

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [7]:
!pip install selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Firefox()

#Abrir navegador

navegador.get("https://www.melhorcambio.com/dolar-hoje")

#Extrair cotação dólar

cotacao_dolar = navegador.find_element(By. XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_dolar = cotacao_dolar.replace(",", ".")

print(cotacao_dolar)

#Extrair cotação euro

navegador.get("https://www.melhorcambio.com/euro-hoje")

cotacao_euro = navegador.find_element(By. XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_euro = cotacao_euro.replace(",", ".")

print(cotacao_euro)

#Extrair cotação Yen

navegador.get("https://www.melhorcambio.com/iene-hoje")

cotacao_yen = navegador.find_element(By. XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_yen = cotacao_yen.replace(",", ".")

print(cotacao_yen)

#Extrair cotação Ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By. XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)

Defaulting to user installation because normal site-packages is not writeable
4.93
5.30
0.0340
320.42


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [8]:
import pandas as pd

tbl = pd.read_excel('Produtos.xlsx')
display(tbl)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [11]:
# Passo 5: Recalcular o preço de cada produto
tbl.loc[tbl["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tbl.loc[tbl["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tbl.loc[tbl["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)
tbl.loc[tbl["Moeda"] == "yen", "Cotação"] = float(cotacao_yen)

# atualizar o preço base reais (preço base original * cotação)
tbl["Preço de Compra"] = tbl["Preço Original"] * tbl["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tbl["Preço de Venda"] = tbl["Preço de Compra"] * tbl["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tbl)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.93,4929.9507,1.40,6901.93098
1,Carro Renault,4500.00,Euro,5.30,23850.0000,2.00,47700.00000
2,Notebook Dell,899.99,Dólar,4.93,4436.9507,1.70,7542.81619
3,IPhone,799.00,Dólar,4.93,3939.0700,1.70,6696.41900
4,Carro Fiat,3000.00,Euro,5.30,15900.0000,1.90,30210.00000
5,Celular Xiaomi,480.48,Dólar,4.93,2368.7664,2.00,4737.53280
6,Joia 20g,20.00,Ouro,320.42,6408.4000,1.15,7369.66000


### Agora vamos exportar a nova base de preços atualizada

In [13]:
# Passo 6: Salvar os novos preços dos produtos
tbl.to_excel("Produtos Novo.xlsx", index=False)